# Linkedin Public Website Information Crawler/Spider

In [13]:
import scrapy
import json
import pandas as pd
import time
import langchain as Langchain

class Linkedin_Public_Spider(scrapy.Spider):
    name = "linkedin_company_profile"

    # Hier kommen alle öffentlichen LinkedIn-Seiten der Cryptofundraising-Seite hin, die mit einem anderen Crawler abgerufen werden
    public_profile_webpages = [
        'https://www.linkedin.com/company/kryptoskatt/about/',
        # Weitere URLs hier hinzufügen
    ]

    custom_settings = {
        'FEEDS': {
            'Linkedin_public_profiles_output.csv': {
                'format': 'csv',
                'fields': ['name', 'summary', 'industry', 'size', 'founded'],
            },
        },
        'DOWNLOADER_MIDDLEWARES': {
            'scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware': None,
            'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware': None,
            'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware': None,
            'scrapy.downloadermiddlewares.retry.RetryMiddleware': None,
            'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware': None,
            'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware': None,
            'scrapy.downloadermiddlewares.cookies.CookiesMiddleware': None,
            'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware': None,
            'scrapy.downloadermiddlewares.stats.DownloaderStats': None,
            'scrapy_user_agents.middlewares.RandomUserAgentMiddleware': 400,
            'scrapy_proxies.RandomProxy': 100,
            'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware': 110,
        },
        'USER_AGENT_LIST': 'path/to/user_agent_list.txt',  # Pfad zur Datei mit User-Agent-Liste
        'RETRY_TIMES': 5,  # Anzahl der Wiederholungsversuche bei Fehlern
        'PROXY_POOL_ENABLED': True,  # Aktiviert die Verwendung von Proxies
        'REQUEST_FINGERPRINTER_IMPLEMENTATION': 'scrapy.utils.request.fingerprint.request_fingerprint',
    }

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.langchain = Langchain(api_key='YOUR_API_KEY', endpoint='YOUR_API_ENDPOINT')

    def start_requests(self):
        for index, url in enumerate(self.public_profile_webpages):
            yield scrapy.Request(url=url, callback=self.parse_response, meta={'public_profile_index': index})

    def parse_response(self, response):
        public_profile_index = response.meta['public_profile_index']
        print('***************')
        print('****** Scraping page ' + str(public_profile_index+1) + ' of ' + str(len(self.public_profile_webpages)))
        print('***************')

        firmen_items = {}

        # Extrahieren der gewünschten Informationen
        firmen_items['name'] = response.css('.top-card-layout__entity-info h1::text').get(default='not-found').strip()
        firmen_items['summary'] = response.css('.top-card-layout__entity-info h4 span::text').get(default='not-found').strip()

        try:
            firmen_details = response.css('.core-section-container__content .mb-2')

            firma_branchentyp = firmen_details[1].css('.text-md::text').getall()
            firmen_items['industry'] = firma_branchentyp[1].strip()

            größe_der_firma = firmen_details[2].            größe_der_firma = firmen_details[2].css('.text-md::text').getall()
            firmen_items['size'] = größe_der_firma[1].strip()

            gründung_der_firma = firmen_details[5].css('.text-md::text').getall()
            firmen_items['founded'] = gründung_der_firma[1].strip()
        
        except IndexError:
            print("Error: Firma wurde ausgelassen - Einige Infos fehlen!")

        # Konvertieren des firmen_items in JSON
        firmen_json = json.dumps(firmen_items)

        # Aufrufen der Langchain-API für die Analyse
        analysis = self.langchain.analyze(firmen_json)

        # Extrahieren der analysierten Informationen
        sentiment = analysis['sentiment']
        entities = analysis['entities']
        keywords = analysis['keywords']

        # Hinzufügen der analysierten Informationen zu firmen_items
        firmen_items['sentiment'] = sentiment
        firmen_items['entities'] = entities
        firmen_items['keywords'] = keywords

        yield firmen_items

        public_profile_index = public_profile_index + 1

        if public_profile_index < len(self.public_profile_webpages):
            next_url = self.public_profile_webpages[public_profile_index]
            time.sleep(20)  # Verzögerung von 20 Sekunden einfügen
            yield scrapy.Request(url=next_url, callback=self.parse_response, meta={'public_profile_index': public_profile_index})



In [ ]:
# im Terminal:
# 1) Folder erstellen:
# cd "C:\Users\Engineer\OneDrive - DataCraft GmbH\Dokumente\DataCraft\4.1. Vorbereitung Praxis\Webcrawler"

# 2) Scrapy Projekt anlegen
# scrapy startproject Linkedin_Firmeninfos_public

# 3) in Projektverzeichnis navigieren:
# cd Linkedin_Firmeninfos_public
# Wenn Projekt schon angelegt ist, dann:
# cd C:\Users\Engineer\"OneDrive - DataCraft GmbH"\Dokumente\DataCraft\"4.1. Vorbereitung Praxis"\Webcrawler\Linkedin_Firmen_infos

# 4) Spider erstellen
# scrapy genspider Linkedin_Public_Spider linkedin.com

# 5) Spider laufen lassen
# scrapy runspider Linkedin_Public_Spider.py
# mit output als csv:
# scrapy runspider Linkedin_Public_Spider.py -o Linkedin_public_profiles_output.csv


In [ ]:
# Brauche Langchain, um nicht geblockt zu werden:
# Terminal: pip install langchain
# API ist kostenpflichtig